# Форматы данных

Материалы:
* Макрушин С.В. "Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.h5py.org/en/stable/
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

2. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

3. Создайте 2 матрицы размера 1000x1000, используя различные параметризируемые распределения из numpy (https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.random.html#distributions)

После этого сохраните получившиеся матрицы в hdf5-файл в виде двух различных датасетов. В качестве описания каждого датасета укажите параметры используемых распределений 

## Лабораторная работа №3

### JSON

1.1 Считайте файл `contributors_sample.json`, воспользовавшись модулем `json` и свяжите загруженные данные с переменной `contributors`. Выведите на экран уникальные почтовые домены, содержащиеся в почтовых адресах людей. Под доменом понимается часть адреса, следующая за символом `@`.

In [74]:
with open(
    'contributors_sample.json', 
    'r',
    encoding='utf-8'
) as fp:
    contributors = json.load(fp)
#contributors

In [75]:
mails = []
a = []
for person in contributors:
    mails.append(person['mail'])
mails
for i in mails:
    m = i.split('@')
    a.append(m)
a = np.array(a)
np.unique(a[:,1])

array(['gmail.com', 'hotmail.com', 'yahoo.com'], dtype='<U20')

1.2 Посчитайте, как часто встречается та или иная должность во всем наборе данных. Выведите на экран 5 должностей, которые встречаются наиболее часто. Для каждого пользователя из `contributors` выясните, какая из его должностей является наиболее распространенной (в смысле частоты упоминания во всем датасете) и добавьте ключ `top_job`, в котором хранится название этой должности.

In [78]:
jobs = []
for person in contributors:
    jobs.extend(ph for ph in person['jobs'])
unique, counts = np.unique(jobs, return_counts=True)
#print(unique, counts)

res = np.column_stack([unique, counts])
ress = res[res[:, 1].argsort()[::-1]][:5]
#print(ress)
###############################


##########################
top_job = []
for person in contributors:
    jb = []
    jb1 = 0
    jb2 = []
    r = 0
    for j in person['jobs']:
        r = res[np.where(res == j),1][:1]
        jb.append(r)
        jb1 = max(jb)
        jb2.append(j)
        k = jb2[jb.index(jb1)]
    top_job.append(k)
#print(top_job)

#dict1 = dict(zip(f, top_job))
for i in range(len(contributors)):
    contributors[i]['top_job'] = top_job[i]
#contributors


1.3 Создайте pd.DataFrame contributors_df, имеющий столбцы id, username и sex и n_jobs. Столбец n_jobs содержит кол-во должностей пользователя. При необходимости вы можете преобразовать исходные данные в списке contributors в удобный для создания pd.DataFrame вид.

Сгруппируйте полученные данные по столбцам n_jobs и sex. Выведите на экран серию pd.Series, в которой содержится информация о количестве человек в каждой группе.

In [38]:
contributors_df = pd.DataFrame(data = contributors)
contributors_df = contributors_df.drop(["name"], axis = 1)
contributors_df = contributors_df.drop(["address"], axis = 1)
contributors_df = contributors_df.drop(["mail"], axis = 1)
contributors_df['n_jobs'] = contributors_df['jobs'].str.len()
contributors_df = contributors_df.groupby(['n_jobs','sex']).count()
f = contributors_df.iloc[:,1]
print(f)
print('//////////')
type(f)

n_jobs  sex
3       F      703
        M      690
4       F      733
        M      704
5       F      700
        M      670
Name: jobs, dtype: int64
//////////


pandas.core.series.Series

### XML

2.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`. Выведите на экран шаги рецепта с id `84797`.

In [16]:
from bs4 import BeautifulSoup

In [17]:
with open(
    'steps_sample.xml', 
    'r',
    encoding='utf-8'
) as fp:
    xml = BeautifulSoup(fp)

In [79]:
#xml

In [1]:
recipess = {}
recipes = xml.find_all('recipe')
for i in range(len(recipes)):
    id = recipes[i].find_all('id')
    steps = recipes[i].find_all('step')
    steps = [steps[j].text for j in range(len(steps))]
    recipess[id[0].text] = recipess.get(id[0].text, steps)
#recipess

NameError: name 'xml' is not defined

In [66]:
with open('steps_sample.json', 'w') as fp:
    json.dump(recipess, fp)

In [81]:
with open(
    'steps_sample.json', 
    'r',
    encoding='utf-8'
) as fp:
    steps_sample = json.load(fp)
#steps_sample

In [65]:
recipess['84797']

['honey mustard sauce: whisk all the ingredients together serve warm or cold',
 'easy bbq sauce: combine all ingredients in a pot& cook over low heat until the sugar is dissolved',
 'serve warm or cold',
 'garlic dill sauce: mix all the ingredients and chill until ready to serve']

2.2 Получите список идентификаторов рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов. Выведите на экран количество таких рецептов.

In [23]:
rec = set()

recipes = xml.find_all('recipe')
def get_key(d, value):  #узнаёт ключ, словаря с известным значением 
    for k, v in d.items():
        if v == value:
            return k

for i in range(len(recipes)):
    idd = recipes[i].find_all('id')
    steps = recipes[i].find_all('step')
    for step in steps:
        if (get_key(step.attrs, '1') ==  'has_minutes') or (get_key(step.attrs, '1') == 'has_hours'):
            rec.add(idd[0].text)
len(rec)

22437

2.3 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений. При решении задачи не используйте метод `iterrows` и аналогичные ему, позволяющие итерироваться по таблице построчно.

Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [24]:
recipes = pd.read_csv('recipes_sample.csv', sep = ',')
recipes = recipes.set_index('id')
recipes

,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
id,,,,,,,
44123,george s at the cove black bean soup,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
67664,healthy for them yogurt popsicles,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
38798,i can t believe it s spinach,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
35173,italian gut busters,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
84797,love is in the air beef fondue sauces,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...
267661,zurie s holey rustic olive and cheddar bread,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0
386977,zwetschgenkuchen bavarian plum cake,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0
103312,zwiebelkuchen southwest german onion cake,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN


In [82]:
dd= []
file = open("steps_sample.xml","r").read()
soup = BeautifulSoup(file,'xml')
for recipe in soup.find_all('recipe'):
    recipe_id= int(recipe.find("id").get_text())
    steps_count= len(recipe.find_all("step"))
    dd.append(steps_count)
#dd

In [39]:
recipes['n_steps'].where(recipes['n_steps'].isnull(), dd, inplace = True)

In [83]:
#recipes

In [41]:
recipes['n_steps'].isnull().sum()

0

In [42]:
recipes['n_steps'].astype('int')

id
44123     11
67664      3
38798      5
35173      7
84797      4
          ..
267661    16
386977    22
103312    10
486161     7
298512     9
Name: n_steps, Length: 30000, dtype: int32

In [43]:
recipes.to_csv('recipes_sample_with_filled_nsteps.csv')

### hdf

3.1 Выведите названия всех датасетов, находящихся в файле `nutrition_sample.h5`, а также размерность матриц, содержащихся в данных датасетах и их метаданные.

Формат вывода:
```
Dataset name=dataset_0 n_rows=30000 n_cols=2 col_0=recipe_id col_1=calories (#)
Dataset name=dataset_1 n_rows=30000 n_cols=2 col_0=recipe_id col_1=total fat (PDV)
Dataset name=dataset_2 n_rows=30000 n_cols=2 col_0=recipe_id col_1=sugar (PDV)
...
```

In [71]:
dataset = {}
with h5py.File("nutrition_sample.h5", "r") as hf:
    for key in hf.keys():
        file = hf[key]
        metadata = dict(file.attrs)
        print(f"Dataset name={key} n_rows={file.shape[0]} n_cols={file.shape[1]} col_0={metadata['col_0']} col_1={metadata['col_1']} ")
        dataset[key] = {'all_data' : {'data': file[:]}, 'attrs': metadata}


<HDF5 dataset "dataset_0": shape (30000, 2), type "<f8">
{'col_0': 'recipe_id', 'col_1': 'calories (#)'}
Dataset name=dataset_0 n_rows=30000 n_cols=2 col_0=recipe_id col_1=calories (#) 
<HDF5 dataset "dataset_1": shape (30000, 2), type "<f8">
{'col_0': 'recipe_id', 'col_1': 'total fat (PDV)'}
Dataset name=dataset_1 n_rows=30000 n_cols=2 col_0=recipe_id col_1=total fat (PDV) 
<HDF5 dataset "dataset_2": shape (30000, 2), type "<f8">
{'col_0': 'recipe_id', 'col_1': 'sugar (PDV)'}
Dataset name=dataset_2 n_rows=30000 n_cols=2 col_0=recipe_id col_1=sugar (PDV) 
<HDF5 dataset "dataset_3": shape (30000, 2), type "<f8">
{'col_0': 'recipe_id', 'col_1': 'sodium (PDV)'}
Dataset name=dataset_3 n_rows=30000 n_cols=2 col_0=recipe_id col_1=sodium (PDV) 
<HDF5 dataset "dataset_4": shape (30000, 2), type "<f8">
{'col_0': 'recipe_id', 'col_1': 'protein (PDV)'}
Dataset name=dataset_4 n_rows=30000 n_cols=2 col_0=recipe_id col_1=protein (PDV) 
<HDF5 dataset "dataset_5": shape (30000, 2), type "<f8">
{'col_0

In [66]:
dataset.items()

dict_items([('dataset_0', {'all_data': {'data': array([[4.41230e+04, 8.04700e+02],
       [6.76640e+04, 1.64600e+02],
       [3.87980e+04, 5.38000e+01],
       ...,
       [1.03312e+05, 8.64100e+02],
       [4.86161e+05, 4.15200e+02],
       [2.98512e+05, 1.88000e+02]])}, 'attrs': {'col_0': 'recipe_id', 'col_1': 'calories (#)'}}), ('dataset_1', {'all_data': {'data': array([[4.41230e+04, 1.08000e+02],
       [6.76640e+04, 3.00000e+00],
       [3.87980e+04, 5.00000e+00],
       ...,
       [1.03312e+05, 8.70000e+01],
       [4.86161e+05, 2.60000e+01],
       [2.98512e+05, 1.10000e+01]])}, 'attrs': {'col_0': 'recipe_id', 'col_1': 'total fat (PDV)'}}), ('dataset_2', {'all_data': {'data': array([[4.41230e+04, 2.60000e+01],
       [6.76640e+04, 5.00000e+00],
       [3.87980e+04, 2.00000e+00],
       ...,
       [1.03312e+05, 3.00000e+01],
       [4.86161e+05, 3.40000e+01],
       [2.98512e+05, 5.70000e+01]])}, 'attrs': {'col_0': 'recipe_id', 'col_1': 'sugar (PDV)'}}), ('dataset_3', {'all_dat

3.2 Разбейте каждый из имеющихся датасетов на две части: 1 часть содержит только те строки, где PDV (Percent Daily Value) превышает 100%; 2 часть содержит те строки, где PDV составляет не более 100%. Создайте 2 группы в файле и разместите в них соответствующие части датасета c сохранением метаданных исходных датасетов. Итого должно получиться 2 группы, содержащие несколько датасетов. Датасеты, которые не содержат информацию о PDV, оставьте вне групп. Сохраните результаты в файл `nutrition_grouped.h5`.

In [73]:
dataset= {data : info for data, info in dataset.items() if "PDV" in info["attrs"]["col_1"]}

update_dataset= {"PDV>100" : {}, "PDV<100": {}}
for datasets, val in dataset.items():
    data= val["all_data"]["data"]
    update_dataset["PDV>100"][datasets] = {"data" :  data, "attrs" : val["attrs"], "name": datasets}
    update_dataset["PDV<100"][datasets] = {"data" :  data, "attrs" : val["attrs"], "name": datasets}
    
for datasets, data in update_dataset.items():
    print(update_dataset)
    print(data)

{'PDV>100': {'dataset_1': {'data': array([[4.41230e+04, 1.08000e+02],
       [6.76640e+04, 3.00000e+00],
       [3.87980e+04, 5.00000e+00],
       ...,
       [1.03312e+05, 8.70000e+01],
       [4.86161e+05, 2.60000e+01],
       [2.98512e+05, 1.10000e+01]]), 'attrs': {'col_0': 'recipe_id', 'col_1': 'total fat (PDV)'}, 'name': 'dataset_1'}, 'dataset_2': {'data': array([[4.41230e+04, 2.60000e+01],
       [6.76640e+04, 5.00000e+00],
       [3.87980e+04, 2.00000e+00],
       ...,
       [1.03312e+05, 3.00000e+01],
       [4.86161e+05, 3.40000e+01],
       [2.98512e+05, 5.70000e+01]]), 'attrs': {'col_0': 'recipe_id', 'col_1': 'sugar (PDV)'}, 'name': 'dataset_2'}, 'dataset_3': {'data': array([[4.41230e+04, 1.90000e+01],
       [6.76640e+04, 1.00000e+00],
       [3.87980e+04, 3.00000e+00],
       ...,
       [1.03312e+05, 1.80000e+01],
       [4.86161e+05, 2.60000e+01],
       [2.98512e+05, 1.10000e+01]]), 'attrs': {'col_0': 'recipe_id', 'col_1': 'sodium (PDV)'}, 'name': 'dataset_3'}, 'datase

In [6]:
with h5py.File('nutrition_grouped.h5', 'w') as f:
    for datasets, data in update_dataset.items():
            gg = f.create_group(datasets)
            for j in data:
                data1 = gg.create_dataset(name = data[j]['name'], data = data[j]['data'])
                data1.attrs.update(data[j]['attrs'])

In [30]:
with h5py.File(
    'nutrition_grouped.h5',
    'r'
    ) as hdf:
    #print(hdf.items())
    for key in hdf.keys():
        file = hdf[key]
        metadata = dict(file.attrs)
        print(file['dataset_3'])
        print('Dataset',file['dataset_1'][:])

<HDF5 dataset "dataset_3": shape (30000, 2), type "<f8">
Dataset [[4.41230e+04 1.08000e+02]
 [6.76640e+04 3.00000e+00]
 [3.87980e+04 5.00000e+00]
 ...
 [1.03312e+05 8.70000e+01]
 [4.86161e+05 2.60000e+01]
 [2.98512e+05 1.10000e+01]]
<HDF5 dataset "dataset_3": shape (30000, 2), type "<f8">
Dataset [[4.41230e+04 1.08000e+02]
 [6.76640e+04 3.00000e+00]
 [3.87980e+04 5.00000e+00]
 ...
 [1.03312e+05 8.70000e+01]
 [4.86161e+05 2.60000e+01]
 [2.98512e+05 1.10000e+01]]


3.3 Выведите названия всех групп и датасетов, находящихся в этих группах, из файла `nutrition_grouped.h5` а также размерность матриц, содержащихся в датасетах и их метаданные.

Формат вывода:
```
Dataset name=dataset_0 n_rows=30000 n_cols=2 col_0=recipe_id col_1=calories (#)
Group less_equal_than_100:
    Dataset name=less_equal_than_100/dataset_1 n_rows=28264 n_cols=2 col_0=recipe_id col_1=total fat (PDV)
    ....
Group more_than_100
    Dataset name=more_than_100/dataset_1 n_rows=1736 n_cols=2 col_0=recipe_id col_1=total fat (PDV)
    ....
....
```